### Make required imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from interpret import show
from interpret import data
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree
from interpret.perf import RegressionPerf

### Load dataframe and separate feature and target sets

In [ ]:
df = pd.read_pickle('COS_Seesaw_dataframe.pkl')
print(df)
columns = list(df.columns)
if 'COS_JFJ' in columns:
    columns.remove('COS_JFJ')
else:
    print('Error, target column not in dataframe')
    quit()

if 'time' in columns:
    columns.remove('time')

x = df[columns]
y = df['COS_JFJ']

### Make test train split, random seed is to ensure consistent results on multiple runs
### change  seed value to get different split

In [ ]:
seed = 1
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=seed)

print(len(X_train))
print(len(y_train))

print(len(X_test))
print(len(y_test))

### Show Marginal

In [ ]:
marginal = data.Marginal()
marginal_explanation = marginal.explain_data(X_train, y_train)
show(marginal_explanation)

### Train Models

In [ ]:
ebm = ExplainableBoostingRegressor(random_state=seed, interactions=3)
ebm.fit(X_train, y_train)

### What model learned

In [ ]:
ebm_global = ebm.explain_global(name='EBM')

### Explanation of individul prediction

In [ ]:
ebm_local = ebm.explain_local(X_test, y_test, name='EBM')

### Evaluate performance

In [ ]:
ebm_perf = RegressionPerf(ebm.predict).explain_perf(X_test, y_test, name='EBM')

### Everything at once

In [ ]:
show([marginal_explanation, ebm_global, ebm_perf, ebm_local])